# DNN colon dataset

In [ ]:
import math
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from tqdm import tqdm

from deap import base  # Estructura que permite agrupar todos los componentes de nuestro algoritmo en una misma bolsa
from deap import creator  # Permite crear los componentes de nuestro algoritmo
from deap import tools  # Contiene funciones precargadas

from joblib import Parallel, delayed

# PARTICIONO LOS DATOS

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('colon.csv', header=None)

X_trn, X_tst = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)  # 70% train - 30% validation (esto es, sobre el 80 de arriba)

X_trn.to_csv('data_trn.csv', header=None, index=None)
X_tst.to_csv('data_tst.csv', header=None, index=None)

# Leemos los datasets

In [ ]:
TRAIN = pd.read_csv('data_trn.csv', header=None)
TEST = pd.read_csv('data_tst.csv', header=None)

TRAIN.head(5)

# Normalizamos los datos

In [ ]:
scaler = StandardScaler()

TRAIN = TRAIN.to_numpy()
TEST = TEST.to_numpy()

X_TRAIN = TRAIN[:,:-1]
y_train = (TRAIN[:,-1] + 1) /2

X_TEST = TEST[:,:-1]
y_test = (TEST[:,-1] + 1)/2

scaler.fit(X_TRAIN)

Xtrain = scaler.transform(X_TRAIN)
Xtest = scaler.transform(X_TEST)

In [ ]:
Xtrain.shape


(49, 2000)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

In [ ]:
# Convertir a tensores de PyTorch
X_train = torch.FloatTensor(Xtrain)
y_train = torch.LongTensor(y_train)
X_test = torch.FloatTensor(Xtest)
y_test = torch.LongTensor(y_test)

# Crear DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)


In [ ]:
# Definir la red neuronal
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2000, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Crear una instancia de la red
net = Net()


In [ ]:
# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# Entrenar la red
for epoch in range(1000):  # 100 épocas
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluar la red
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy: %d %%' % (100 * correct / total))


Accuracy: 69 %
